In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Check if TensorFlow can access the GPU
if tf.config.list_physical_devices('GPU'):
    print("GPU available")
else:
    print("GPU not available")

# Define the paths to your dataset
data_dir = '/content/drive/MyDrive/Laboratory'
positive_dir = os.path.join(data_dir, 'Positive')
negative_dir = os.path.join(data_dir, 'Negative')

# Create lists to store file paths and labels
file_paths = []
labels = []

# Read the Positive images
for file in os.listdir(positive_dir):
    file_paths.append(os.path.join(positive_dir, file))
    labels.append(1)  # Label for positive images

# Read the Negative images
for file in os.listdir(negative_dir):
    file_paths.append(os.path.join(negative_dir, file))
    labels.append(0)  # Label for negative images

# Convert to numpy arrays
file_paths = np.array(file_paths)
labels = np.array(labels)

# Convert labels to a list of strings
labels = labels.astype(str).tolist()

# Split the data into training and testing sets
train_paths, test_paths, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=42)

# ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Helper function to create a data generator
def create_generator(file_paths, labels, datagen, batch_size=32):
    return datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': file_paths, 'class': labels}),
        x_col='filename',
        y_col='class',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

# Create generators for training and testing
train_generator = create_generator(train_paths, train_labels, train_datagen)
test_generator = create_generator(test_paths, test_labels, test_datagen, batch_size=1)  # Set batch_size to 1 for testing

# Load the NASNetMobile model with pre-trained ImageNet weights
base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout layer for regularization
predictions = Dense(1, activation='sigmoid')(x)

# Create the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Learning rate scheduler and early stopping callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Train the model with callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=30,
    validation_data=test_generator,
    validation_steps=len(test_generator),
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model on the test set
test_labels = np.array(test_labels)
test_labels = test_labels.astype(int)
test_generator.reset()
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)
predictions = (predictions > 0.5).astype(int).flatten()

# Calculate performance metrics
print("Classification Report:")
print(classification_report(test_labels, predictions, target_names=['Negative', 'Positive']))


GPU not available
Found 384 validated image filenames belonging to 2 classes.
Found 96 validated image filenames belonging to 2 classes.
Epoch 1/30
12/12 [==============================] - 311s 17s/step - loss: 0.1020 - accuracy: 0.9401 - val_loss: 5.4447e-06 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/30
12/12 [==============================] - 191s 16s/step - loss: 0.0797 - accuracy: 0.9922 - val_loss: 0.0022 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/30
12/12 [==============================] - 194s 16s/step - loss: 0.0731 - accuracy: 0.9844 - val_loss: 6.8132e-07 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/30
12/12 [==============================] - 194s 16s/step - loss: 0.0179 - accuracy: 0.9974 - val_loss: 4.0243e-05 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/30
12/12 [==============================] - 194s 16s/step - loss: 6.5999e-06 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/30
12/12 [==============================] - 190s 16s/step - loss

In [ ]:
from sklearn.metrics import confusion_matrix , accuracy_score, precision_score, recall_score
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}')

Accuracy: 46.88%
Precision: 25.00%
Recall: 4.26
